# Demo 0: Example and usage

In order to make things simple the following rules have been followed
during development:

-   `deel-lip` follows the `keras` package structure.
-   All elements (layers, activations, initializers, ...) are compatible
    with standard the `keras` elements.
-   When a k-Lipschitz layer overrides a standard keras layer, it uses
    the same interface and the same parameters. The only difference is a
    new parameter to control the Lipschitz constant of a layer.

## Which layers are safe to use?

The following table indicates which layers are safe to use in a Lipshitz
network, and which are not.

| layer                                                                                         | 1-lip? | deel-lip equivalent                                                                                         | comments                                                                          |
|-----------------------------------------------------------------------------------------------|--------|-------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------|
| `Dense`                                                                                       | no     | `SpectralDense`<br>`FrobeniusDense`                       | `SpectralDense` and `FrobeniusDense` are similar when there is a single output. |
| `Conv2D`                                                                                      | no     | `SpectralConv2D`<br>`FrobeniusConv2D`                     | `SpectralConv2D` also implements Björck normalization.                           |
| `MaxPooling`<br>`GlobalMaxPooling`             | yes    | n/a                                                                                                         |                                                                                   |
| `AveragePooling2D`<br>`GlobalAveragePooling2D` | no     | `ScaledAveragePooling2D`<br>`ScaledGlobalAveragePooling2D` | The lipschitz constant is bounded by `sqrt(pool_h * pool_h)`.                     |
| `Flatten`                                                                                     | yes    | n/a                                                                                                         |                                                                                   |
| `Dropout`                                                                                     | no     | None                                                                                                        | The lipschitz constant is bounded by the dropout factor.                          |
| `BatchNormalization`                                                                          | no     | None                                                                                                        | We suspect that layer normalization already limits internal covariate shift.      |

## Design tips

Designing lipschitz networks requires a careful design in order to avoid
vanishing/exploding gradient problems.

Choosing pooling layers:

| layer                                                                            | advantages                                                                   | disadvantages                                                                      |
|----------------------------------------------------------------------------------|------------------------------------------------------------------------------|------------------------------------------------------------------------------------|
| `ScaledAveragePooling2D` and `MaxPooling2D`                                      | very similar to original implementation (just add a scaling factor for avg). | not norm preserving nor gradient norm preserving.                                  |
| `InvertibleDownSampling`                                                         | norm preserving and gradient norm preserving.                                | increases the number of channels (and the number of parameters of the next layer). |
| `ScaledL2NormPooling2D` (_sqrt(avgpool(x\*\*2))_) | norm preserving.                                                             | lower numerical stability of the gradient when inputs are close to zero.           |

Choosing activations:

| layer                                                                  | advantages                                                                                   | disadvantages                                                                                  |
|------------------------------------------------------------------------|----------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------|
| `ReLU`                                                                 |                                                                                              | create a strong vanishing gradient effect. If you manage to learn with it, please call 911.    |
| `MaxMin` (_stack(\[ReLU(x), ReLU(-x)\])_) | have similar properties to ReLU, but is norm and gradient norm preserving                    | double the number of outputs                                                                   |
| `GroupSort`                                                            | Input and GradientNorm preserving. Also limit the need of biases (as it is shift invariant). | more computationally expensive, (when its parameter _n_ is large) |

Please note that when learning with the `HKR_loss` and `HKR_multiclass_loss`, no
activation is required on the last layer.


### How to use it ?
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deel-ai/deel-lip/blob/master/docs/notebooks/demo0.ipynb)

Here is an example of 1-lipschitz network trained on MNIST:

In [1]:
from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from deel.lip.model import Sequential
from deel.lip.activations import GroupSort
from deel.lip.losses import MulticlassHKR, MulticlassKR
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np

# Sequential (resp Model) from deel.model has the same properties as any lipschitz model.
# It act only as a container, with features specific to lipschitz
# functions (condensation, vanilla_exportation...) but The layers are fully compatible
# with the tf.keras.model.Sequential/Model
model = Sequential(
    [
        Input(shape=(28, 28, 1)),
        # Lipschitz layers preserve the API of their superclass ( here Conv2D )
        # an optional param is available: k_coef_lip which control the lipschitz
        # constant of the layer
        SpectralConv2D(
            filters=16,
            kernel_size=(3, 3),
            activation=GroupSort(2),
            use_bias=True,
            kernel_initializer="orthogonal",
        ),
        # usual pooling layer are implemented (avg, max...), but new layers are also available
        ScaledL2NormPooling2D(pool_size=(2, 2), data_format="channels_last"),
        SpectralConv2D(
            filters=16,
            kernel_size=(3, 3),
            activation=GroupSort(2),
            use_bias=True,
            kernel_initializer="orthogonal",
        ),
        ScaledL2NormPooling2D(pool_size=(2, 2), data_format="channels_last"),
        # our layers are fully interoperable with existing keras layers
        Flatten(),
        SpectralDense(
            32,
            activation=GroupSort(2),
            use_bias=True,
            kernel_initializer="orthogonal",
        ),
        FrobeniusDense(
            10, activation=None, use_bias=False, kernel_initializer="orthogonal"
        ),
    ],
    # similary model has a parameter to set the lipschitz constant
    # to set automatically the constant of each layer
    k_coef_lip=1.0,
    name="hkr_model",
)

# HKR (Hinge-Krantorovich-Rubinstein) optimize robustness along with accuracy
model.compile(
    # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
    # note also in the case of lipschitz networks, more robustness require more parameters.
    loss=MulticlassHKR(alpha=50, min_margin=0.05),
    optimizer=Adam(1e-3),
    metrics=["accuracy", MulticlassKR()],
)

model.summary()

# load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# standardize and reshape the data
x_train = np.expand_dims(x_train, -1)
mean = x_train.mean()
std = x_train.std()
x_train = (x_train - mean) / std
x_test = np.expand_dims(x_test, -1)
x_test = (x_test - mean) / std
# one hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# fit the model
model.fit(
    x_train,
    y_train,
    batch_size=2048,
    epochs=30,
    validation_data=(x_test, y_test),
    shuffle=True,
)

# once training is finished you can convert
# SpectralDense layers into Dense layers and SpectralConv2D into Conv2D
# which optimize performance for inference
vanilla_model = model.vanilla_export()

2024-09-06 14:56:17.825542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-06 14:56:17.837283: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-06 14:56:17.840865: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-06 14:56:17.849389: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-06 14:56:19.159295: W tensorflow/compiler/tf2

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ spectral_conv2d                 │ (None, 28, 28, 16)     │           321 │
│ (SpectralConv2D)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ scaled_l2_norm_pooling2d        │ (None, 14, 14, 16)     │             0 │
│ (ScaledL2NormPooling2D)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_conv2d_1               │ (None, 14, 14, 16)     │         4,641 │
│ (SpectralConv2D)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ scaled_l2_norm_pooling2d_1      │ (None, 7, 7, 16)       │             0 │
│ (ScaledL2NormPooling2D)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense (SpectralDense)  │ (None, 32)             │        50,241 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ frobenius_dense                 │ (None, 10)             │           640 │
│ (FrobeniusDense)                │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 55,843 (218.14 KB)

 Trainable params: 27,920 (109.06 KB)

...

Epoch 1/30


I0000 00:00:1725627382.892541  863830 service.cc:146] XLA service 0x55fae9023e40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725627382.892568  863830 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 SUPER, Compute Capability 7.5
2024-09-06 14:56:22.938074: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-06 14:56:23.125054: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902


16/30 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - MulticlassKR: 0.0607 - accuracy: 0.1946 - loss: 6.8997

I0000 00:00:1725627386.455230  863830 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


30/30 ━━━━━━━━━━━━━━━━━━━━ 9s 139ms/step - MulticlassKR: 0.1052 - accuracy: 0.3134 - loss: 4.9761 - val_MulticlassKR: 0.2972 - val_accuracy: 0.8299 - val_loss: 0.3718
Epoch 2/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - MulticlassKR: 0.3217 - accuracy: 0.8469 - loss: 0.2719 - val_MulticlassKR: 0.3882 - val_accuracy: 0.9084 - val_loss: 0.0118
Epoch 3/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - MulticlassKR: 0.4049 - accuracy: 0.9042 - loss: -0.0177 - val_MulticlassKR: 0.4820 - val_accuracy: 0.9280 - val_loss: -0.1498
Epoch 4/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - MulticlassKR: 0.5051 - accuracy: 0.9237 - loss: -0.1753 - val_MulticlassKR: 0.6148 - val_accuracy: 0.9410 - val_loss: -0.3100
Epoch 5/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - MulticlassKR: 0.6573 - accuracy: 0.9358 - loss: -0.3483 - val_MulticlassKR: 0.8202 - val_accuracy: 0.9465 - val_loss: -0.5050
Epoch 6/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - MulticlassKR: 0.8781 - accuracy: 0.9404 - loss: -0.5443 - val_Mult